In [1]:
import pandas as pd
from os import listdir

#a
#
# Set options & Paths
pd.set_option('display.max_columns', None)
directory_path = r'C:\Users\FILAB\Desktop\230711_자동차 산업'
#

EV_df = pd.read_excel(directory_path + "\\230720_EV_df.xlsx")
# EV_battery_df['텍스트'] = EV_battery_df['발명의 명칭'] + " " + EV_battery_df['요약'] + " " + EV_battery_df['대표청구항']

In [ ]:
EV_df = EV_df.iloc[:,1:]
EV_df

In [ ]:
def resumetable(df, max_unique=7):
    print(f'데이터셋 형상: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns = ['데이터 타입'])
    summary = summary.reset_index()
    summary = summary.rename(columns = {'index': '피처'})
    summary['결측값 개수'] = df.isnull().sum().values
    summary['고윳값 개수'] = df.nunique().values
    summary['첫 번째 값'] = df.loc[0].values
    summary['두 번째 값'] = df.loc[1].values
    summary['세 번째 값'] = df.loc[2].values
    summary['고윳값예시'] = df.apply(lambda x: list(x.unique()) if x.nunique() <= max_unique else ', '.join(str(val) for val in list(x.unique()[:4]) + ['...']), axis=0).values


    return summary

def print_unique(summary):
    for index, row in summary.iterrows():
        print(f"{row['피처']} : {row['고윳값 예시']}")

resume_df = resumetable(EV_df)

In [ ]:
resume_df


In [ ]:
EV_df['Current CPC Main 4 digit'] = EV_df['Current CPC Main'].str.extract(r'(....)')
EV_df['Current CPC Main 8 digit'] = EV_df['Current CPC Main'].str.extract(r'(.+/)')
EV_df['Current CPC Main 8 digit'] = EV_df['Current CPC Main 8 digit'].str.replace("\/", "")
EV_df['Current CPC Main 8 digit']

In [ ]:
# 인용 자료 확보

Patent_df = pd.read_csv(directory_path + "\\BWD.txt")
Patent_df

temp_df = EV_df.merge(Patent_df, left_on="등록번호", right_on = "PatNum", how= 'left')



temp_df


In [ ]:
citation_df = temp_df.loc[:, ['BWD', '등록번호']].copy()
citation_df.columns = ['source', 'target']

citation_df.dropna(inplace=True)
citation_df 

In [ ]:
EV_patents = set(EV_df['등록번호'])
citation_df = citation_df[citation_df['source'].isin(EV_patents)]
citation_df
# idx_list = []

In [ ]:
del Patent_df

In [ ]:
import pandas as pd
import time


#최원준 데이터 버전

def df_to_kp(df):
    """
    Get Dataframe with Knowledge Persistence scores of each directed path between two nodes(source-target)

    A directed path is a type of graph and can be defined with relations of many nodes
    Knowledge Persistence(KP) is a score that measures the level of influence in the knowledge flow.
    (especially the knowledge flow of patent data)
    It can be the basis of Main Path Analysis.

    ▷ Parameters
    ----------
    source_target : list
        e.g.
        [['A', 'B'],
        ['B', 'C'],
        ['C', 'D']]

    ▷ Returns
    -------
    pd.Dataframe
        columns : 'target', 'kp'

    ▷ Notes
    -----
    ...

    """

    start = time.time()
    # Backward weighting all the edges(citations)
    # target 노드에 대한 1/number of bwd citation
    weight = df.groupby(['target'])['source'].count().reset_index()
    step_1 = df.merge(weight, on="target", how="left")
    step_1.columns = ['source', 'target', 'weight']
    step_1['weight'] = 1 / step_1['weight']

    # Endpoint 찾기
    step_2 = step_1[['source', 'target']]
    step_2 = step_2.groupby(['source'])['target'].count().reset_index()
    step_2.columns = ['target', 'countif']
    step_3 = step_1.merge(step_2, on='target', how='left')
    step_3

    # end point 및 바로 전에 있는 kp 계산 및 새로운 DataFrame 생성
    step_4 = step_3[step_3['countif'].isnull()]  # end point에 대한 선택
    KP = step_4[['target']]  # end point를 찾아서 새로운 DataFrame 생성
    KP = KP.drop_duplicates(['target'])  # end point 중 중복된 것 중 하나만 남김
    KP['kp'] = 0  # endpoint의 kp지수는 0을 추가
    step_4 = step_4.groupby(['source']).sum().reset_index()  # endpoint -1 에 대한 특허는 바로 weight가 해당 kp
    step_4 = step_4[['source', 'weight']]  # kp Dataframe에 추가하기 위한 column 선택
    step_4.columns = ['target', 'kp']  # 위의 line에서 컬럼명 변경
    KP = pd.concat([KP, step_4])  # KP 업데이트
    KP = KP.groupby(['target'])['kp'].sum().reset_index()

    # end point에서
    step_4 = step_3[~step_3['countif'].isnull()]
    step_4 = step_4[['source', 'target', 'weight']]
    step_5 = step_4.groupby(['source'])['target'].count().reset_index()
    step_5.columns = ['target', 'countif']
    step_4 = step_4.merge(step_5, on="target", how="left")
    step_5 = step_4[step_4['countif'].isnull()]
    step_5 = step_5.merge(KP, on='target', how='left')
    step_5['kp'] = step_5['weight'] * step_5['kp']
    step_5 = step_5[['source', 'kp']]
    step_5.columns = ['target', 'kp']
    KP = pd.concat([step_5, KP])
    KP = KP.groupby(['target'])['kp'].sum().reset_index()
    KP
    i = 0
    while True:
        step_4 = step_4[~step_4['countif'].isnull()]
        step_4 = step_4[['source', 'target', 'weight']]
        step_5 = step_4.groupby(['source'])['target'].count().reset_index()
        step_5.columns = ['target', 'countif']
        step_4 = step_4.merge(step_5, on="target", how="left")
        step_5 = step_4[step_4['countif'].isnull()]
        step_5 = step_5.merge(KP, on='target', how='left')
        step_5['kp'] = step_5['weight'] * step_5['kp']
        step_5 = step_5[['source', 'kp']]
        step_5.columns = ['target', 'kp']
        KP = pd.concat([step_5, KP])
        KP = KP.groupby(['target'])['kp'].sum().reset_index()
        i = i + 1
        print(i, " : ", len(step_4))
        if len(step_4) == 0:
            break
    print(KP)
    print("--------------------------------")
    print(step_4.sort_values(['countif'], ascending=False))
    print("--------------------------------")
    print("time :", time.time() - start)
    print("KP type : ", type(KP))

    return KP

def lst_to_kp(edges):


    start = time.time()

    # Convert list into pd.DataFrame
    source = []
    target = []
    for edge in edges:
        source.append(edge[0])
        target.append(edge[1])

    df = pd.DataFrame()
    df['source'] = source
    df['target'] = target

    return df_to_kp(df)


def get_values_df(lst:list, df:pd.DataFrame,col:str)->pd.DataFrame:
    # List를 입력받고
    # df에서 해당 List값에 대응하는 Column의 값을 찾아서
    # List값과 대응되는 값의 목록을 df화 하여 반환
    # 이 때, list의 length와 반환되는 df의 length는 같아야 한다.
    # 엑셀의 vlookup 과 비슷한 느낌    
    if not isinstance(lst,list):
        raise TypeError
    if not isinstance(df, pd.DataFrame):
        raise TypeError
    if not isinstance(col, str):
        raise TypeError
    if len(df.columns) != 2:
        print("데이터프레임 칼럼 수를 2로 맞춰주세요")
        raise ValueError
    if not (col in df.columns):
        print("데이터 프레임에 포함되지 않은 칼럼 이름입니다.")
        raise ValueError

  # 기존의 데이터를 딥카피해서 정리
    node_lst = lst.copy()
    node_df = pd.DataFrame()
    date_df = df.copy()

  # 기존의 노드 목록에 추가정보를 붙임
    node_df[col] = node_lst
    left_df = node_df.merge(date_df, on=col,how="left")
    inner_df= node_df.merge(date_df, on=col,how="inner")
    if len(left_df) != len(inner_df):
        print("데이터프레임에 없는 값이 리스트로 들어갔습니다!")
        raise ValueError
    return inner_df

def prepare_dates(df:pd.DataFrame, date_col:str, return_col:str):
    if not isinstance(df, pd.DataFrame):
        raise TypeError
    if not isinstance(date_col, str):
        raise TypeError

    date_df = df.copy()
    date_df.sort_values(date_col, ascending=False, inplace=True)

    codes_lst = []
    for i in date_df.loc[:,return_col]:
        codes_lst.append(i)
        return codes_lst

In [ ]:
import pandas as pd
import numpy as np

# print(len(df))
# print(len(source_pat),"+", len(sink_pat),"+",len(inter_pat),"+",len(isolated_pat), " = ",len(source_pat)+ len(sink_pat)+len(inter_pat)+len(isolated_pat))

def to_dataframe(st:set):
    if type(st) == list:
        tmp_lst = st.copy()
        if not isinstance(st, list):
            raise TypeError
    elif type(st) == set:
        tmp_lst = list(st)
        if not isinstance(st,set):
            raise TypeError
    else:
        print( "list 혹은 set으로 데이터를 입력해주세요")
        raise ValueError

    df = pd.DataFrame()
    tmp_lst

    df['target'] = tmp_lst

    return df

def get_rank(df:pd.DataFrame):
    if not isinstance(df, pd.DataFrame):
        raise TypeError
    if len(df.columns) != 1:
        print("데이터프레임의 칼럼 갯수는 하나여야 합니다.")
        raise ValueError

    tmp_df = df.copy()
    counter = 0
    tmp_lst = []
    for i in tmp_df.index:
        counter += 1
        tmp_lst.append(counter)
    tmp_df.loc[:,"rank"] = tmp_lst

    return tmp_df

def get_related(edges:pd.DataFrame, nodes:set, search_cl="target", target_cl="source")->list:
    # 두 value의 관계를 정의한 edges 정보가 있고
    # 원하는 한쪽의 정보(nodes)가 있을 때
    # df상에서 해당 nodes와 대응되는 값을 list로 반환하는 함수
    if not isinstance(edges, pd.DataFrame):
        raise TypeError("edges should be a DataFrame")
    if not (isinstance(nodes, set) or isinstance(nodes, list)):
        raise TypeError("nodess should be a set or list")
    if not ((search_cl in edges.columns) or (target_cl in edges.columns)):
        print("분석을 하고 싶은 열(column) 이름을 정확하게 넣어 주십시오")
        raise TypeError

    edges_df = edges.copy()
    nodes_df = to_dataframe(nodes)
    nodes_df.columns = [search_cl]
    edges_df = edges_df.merge(nodes_df, on=search_cl, how='inner')
    tmp_lst = list(set(edges_df[target_cl]))

    return tmp_lst




# Cycle을 기반으로 분석
def get_values_df(lst:list, df:pd.DataFrame,col:str)->pd.DataFrame:
    if not isinstance(lst,list):
        raise TypeError
    if not isinstance(df, pd.DataFrame):
        raise TypeError
    if not isinstance(col, str):
        raise TypeError
    if len(df.columns) != 2:
        print("데이터프레임 칼럼 수를 2로 맞춰주세요")
        raise ValueError
    if not (col in df.columns):
        print("데이터 프레임에 포함되지 않은 칼럼 이름입니다.")
        raise ValueError

  # 기존의 데이터를 딥카피해서 정리
    node_lst = lst.copy()
    node_df = pd.DataFrame()
    date_df = df.copy()

  # 기존의 노드 목록에 추가정보를 붙임
    node_df[col] = node_lst
    left_df = node_df.merge(date_df, on=col,how="left")
    inner_df= node_df.merge(date_df, on=col,how="inner")
    if len(left_df) != len(inner_df):
        print("데이터프레임에 없는 값이 리스트로 들어갔습니다!")
        raise ValueError
    return inner_df

def prepare_dates(df:pd.DataFrame, date_col:str, return_col:str):
    if not isinstance(df, pd.DataFrame):
        raise TypeError
    if not isinstance(date_col, str):
        raise TypeError

    date_df = df.copy()
    date_df.sort_values(date_col, ascending=False, inplace=True)

    codes_lst = []
    for i in date_df.loc[:,return_col]:
        codes_lst.append(i)
    return codes_lst

In [ ]:
def mainpath_findout(EV_df, citation_df, gp_cutoff = 0.3, lp_cutoff = 0.7, condition_1_name = "test", condition_2_name = "test", condition_3_name = "tmp"):
    G = nx.from_pandas_edgelist(citation_df, source='source', target='target', create_using=nx.DiGraph())

    # 순환 구조 확인
    cycles = list(nx.simple_cycles(G))
    while cycles:
        if cycles:  # If there are cycles
            # Remove the first edge of the first cycle
            G.remove_edge(cycles[0][0], cycles[0][1])

            # Check if the graph still contains cycles
            cycles = list(nx.simple_cycles(G))
            if cycles:
                print("The graph still contains cycles:")
                for cycle in cycles:
                    print(cycle)
                print()
            else:
                print("The graph doesn't contain any cycle after removing an edge.")

    # Extract edges from the graph
    edges = G.edges()

    # Convert the edges to a DataFrame
    citation_df = pd.DataFrame(list(edges), columns=['source', 'target'])

    kp_df = df_to_kp(citation_df)

    EV_df = EV_df.merge(kp_df, left_on="등록번호", right_on = "target", how= 'left')

    from_to_lst = [i for i in G.edges]


    g2 = nx.DiGraph()
    g2.add_edges_from(from_to_lst)

    # dag여부 판별

    g2 = nx.DiGraph()
    g2.add_edges_from(from_to_lst)

    print("Directed 여부 : ",g2.is_directed())
    print("Acyclic 여부 : ", nx.is_directed_acyclic_graph(g2))


    EV_df['kp'] = EV_df['kp'].replace(np.nan, 0)
    kp_df = EV_df.loc[:, ['등록번호', 'kp']].copy()
    kp_df.columns = ['target', 'kp']

    node_df = kp_df


    # 레이어를 구성해서 모든 노드들을 레이어에 배치
    # 가장 길이가 긴 경로를 구하고, 이를 기반으로 레이어 설정 
    g2 = nx.DiGraph()
    g2.add_edges_from(from_to_lst)

    longest_path = nx.dag_longest_path(g2)
    print("-"*10,"Longest Path" ,"-"*10)
    display(longest_path)
    print("length of the longest path : ", len(longest_path))

    tmp_df = to_dataframe(longest_path)
    layer_df = get_rank(tmp_df)
    layer_df.columns = ["target", "layer"]



    edges_df = citation_df
    df = EV_df

    #source, sink여부 확인해서 node_df에 붙이기
    source_pat = set(edges_df['source']) - set(edges_df['target'])
    sink_pat = set(edges_df['target']) - set(edges_df['source'])
    connected_pat = set(edges_df['target']) | set(edges_df['source'])
    inter_pat = connected_pat - source_pat - sink_pat
    isolated_pat = set(df['등록번호']) - connected_pat

    node_analsys = [source_pat, sink_pat, inter_pat,isolated_pat]


    tmp_lst =[]
    for i in node_df.loc[:,"target"]:
        if i in source_pat:
            tmp_lst.append("source")
        elif i in sink_pat:
            tmp_lst.append("sink")
        elif i in inter_pat:
            tmp_lst.append("intermediate")
        else:
            tmp_lst.append("isolated")
    node_df.loc[:, 'node_type'] = tmp_lst

    print("layer가 부여된 노드의 갯수 : ", len(layer_df))
    print("connected된 노드의 갯수 : ", len(connected_pat))

    # source에 layer1을 부여하기
    layer_df = to_dataframe(source_pat)
    layer_n = 1
    layer_df.loc[:,"layer"] = layer_n
    layer_df.columns = ['target', 'layer']


    # layer1과 연결된 노드들을 검증하기
    layer_n += 1
    former_layer = source_pat.copy()
    edges = edges_df.copy()

    while True:
      # 이전 레이어와 연결된 노드들을 찾은 다음, 그 노드가 직전 노드만으로 전부 설명이 되면 레이어로 확정하기
        print("")
        print("searching layer number: ", layer_n, " of ", len(longest_path))
        connected_lst = get_related(edges, former_layer, "source", "target") #n번째 레이어와 연결된 노드들 탐색
        print("searched nodes that is connected to ",len(connected_lst))


        not_nexts = []
        counter = 0
        for connected_node in connected_lst: # 찾고싶은 노드들
            counter += 1
    #         print(len(connected_lst)," : ", counter)
            bwd_lst = get_related(edges, [connected_node], "target", "source") # 반대로 source만 찾아서 붙이기
            for bwd_node in bwd_lst:
                if not (bwd_node in former_layer):
                    not_nexts.append(connected_node) # 설명 안되는 노드가 존재?
                    break
        nexts = [i for i in connected_lst if not (i in not_nexts)]

        print("former_layer: ", len(layer_df))
        print("new layer: ", len(nexts))
      #확실해진 다음 레이어들을 추가
        tmp_df = to_dataframe(nexts)
        tmp_df.loc[:,"layer"] = layer_n
        tmp_df.columns = ['target', 'layer']
        layer_df = pd.concat([layer_df, tmp_df], axis=0)
        print("renewed layer: ", len(layer_df), " of ", len(node_df) - len(isolated_pat))

        if len(layer_df) == len(connected_pat):
            print("condition fulfilled")
            break

        if len(layer_df) > len(node_df):
            print("레이어의 누적 갯수가 기존 노드를 뛰어넘었습니다. 재검토 필요")
            raise ValueError

      # 이전 레이어에 있던 항목 연결들 제거
        print("former_connections: ", len(edges))
        tmp = []
        for edge_idx in edges.index:
            if edges.loc[edge_idx, 'source'] in former_layer:
                tmp.append(edge_idx)
        edges.drop(tmp, axis=0, inplace=True)  
        print("renewed_connections: ", len(edges))

        former_layer = nexts.copy()
        layer_n += 1

    # 레이어 정보를 노드에 추가
    node_df = node_df.merge(layer_df,on='target',how='left')
    node_df = node_df.fillna(0)


    # 레이어로 GP를 찾음
    layers = set(node_df['layer'])

    tot_df = pd.DataFrame()
    for layer in layers:
        tmp_df = node_df[node_df['layer'] == layer].copy()
        top_kp = tmp_df.sort_values('kp', ascending=False).iloc[0].kp
        print(top_kp)
        if top_kp == 0:
            tmp_df['lp'] = 0
        else:
            tmp_df['lp'] = tmp_df['kp'] / top_kp
        tot_df = pd.concat([tot_df, tmp_df], ignore_index=True)
    node_df = tot_df.copy()

    top_kp = max(node_df['kp'])
    node_df['gp'] = node_df['kp'] / top_kp
    print("top_kp : ", top_kp)

    #cutoff로 HPP(High Persistence Patent) 판별하기
    gp_tmp = node_df['gp'] > gp_cutoff

    lp_tmp = node_df['lp'] > lp_cutoff


    node_df.loc[:,"is_HPP"] = (gp_tmp | lp_tmp)


    hpp_df = node_df[node_df['is_HPP'] == True].copy()
    edges = edges_df.copy()

    hpps = set(hpp_df.target)
    mainpath = set()
    tmps = set()

    counter = 0
    paths = []
    for hpp in hpps:
        path = []
        path.append(hpp)
        counter += 1
        print(hpp, " : ", len(hpps), "중에 ", counter, " 번째")
        while True:
            print("path 길이 : ",len(path))
            head = path[0]
            hpp_related = get_related(edges, [head], "target", "source")
            if len(hpp_related) != 0:
                backward_top = get_values_df(hpp_related, node_df[['target', 'gp']], 'target').sort_values('gp', ascending=False).iloc[0,0]
                path.insert(0, backward_top)
            else:
          # layer가 source쪽 끝에 도착
                backward_top = ""
                break

        while True:
            print("path 길이 : ",len(path))
            tail = path[-1]
            hpp_related = get_related(edges, [tail], "source", "target")
            if len(hpp_related) != 0:      
                forward_top = get_values_df(hpp_related, node_df[['target', 'gp']], 'target').sort_values('gp', ascending=False).iloc[0,0]
                path.append(forward_top)
            else:
          # layer가 sink쪽 끝에 도착
                forward_top = ""
                break
        paths.append(path)

    elems = set()
    for path in paths:
        for elem in path:
            elems.add(elem)

    print("main path에 포함된 특허들 ",len(elems))

    # node에다가 mainpath정보 추가
    tmp_df = to_dataframe(list(elems))
    tmp_df['is_mainpath'] = True
    tmp_df.columns = ['target', 'is_mainpath']
    node_df = node_df.merge(tmp_df, on='target',how="left")
    node_df['is_mainpath'] = node_df['is_mainpath'] == True

    node_df.sort_values(['is_mainpath','is_HPP'], ascending = False)

    EV_df = EV_df.merge(node_df.drop_duplicates(), left_on = '등록번호', right_on = 'target', how ='left')

    tmp_condition_name1 = str(condition_1_name).replace("/", "")
    tmp_condition_name2 = str(condition_2_name).replace("/", "")
    tmp_condition_name3 = str(condition_3_name).replace("/", "")
    EV_df.to_excel(r'C:\Users\FILAB\Desktop\230711_자동차 산업\230726_EV 분류별 MP\\' + str(tmp_condition_name1)+"_"+str(tmp_condition_name2)+"_"+str(tmp_condition_name3)+'_0.3gp.xlsx')
    print("All Completed")

    return EV_df, node_df


In [ ]:
EV_df['출원연도'] = EV_df['출원일'].fillna("")
EV_df['출원연도'] = EV_df['출원연도'].str.extract(r'(....)')
EV_df = EV_df[EV_df['출원연도'].notna()]
EV_df['출원연도'] = EV_df['출원연도'].astype('int')

EV_df['출원연도']

In [ ]:
EV_df['CPC slash digits'] = EV_df['Current CPC Main'].str.extract(r"(.*?(?=/))")
EV_df['CPC slash digits']

In [ ]:
citation_origin = citation_df.copy()
EV_origin = EV_df.copy()

In [ ]:
tmp_Case = EV_origin[['등록번호', 'Current CPC All']]
tmp_Case = explode(tmp_Case.assign(citation = tmp_Case['Current CPC All'].str.split("|")),'citation')
tmp_Case = tmp_Case[['등록번호', 'citation']]
tmp_Case['citation'] = tmp_Case['citation'].str.replace(" ", "")
tmp_Case['citation'] = tmp_Case['citation'].str.extract(r"(.*?(?=/))")
tmp_Case.drop_duplicates(inplace = True)
tmp_Case.groupby('citation').count().sort_values(by = '등록번호', ascending = False) / len(EV_origin)
tmp_Case = tmp_Case.dropna()
tmp_Case

In [ ]:
import networkx as nx

# 출원연도 선택
itteration_conditions_1 = list(set(EV_origin['출원연도']))
itteration_conditions_1 = [i for i in itteration_conditions_1 if i >= 2000] # 2000년대 이후의 연도만 선택
itteration_conditions_1.sort()

itteration_conditions_2 = set(tmp_Case['citation'])
# itteration_conditions_2 = list(set(EV_origin[class_standard]))
# itteration_conditions_3 = ['US', 'CN']

years_set = set()

EV_list = []
df_list = []

for class_chosen in itteration_conditions_2:
    years_set = set()
    for year_chosen in itteration_conditions_1:
    #         for nation in itteration_conditions_3:            
        years_set.add(year_chosen)
        print("===================New Itteration Starts=================================")
        print(years_set)
        print(class_chosen)
        print()
        citation_df = citation_origin.copy()
        EV_df = EV_origin.copy()

        EV_df = EV_df[EV_df['Current CPC All'].str.find(class_chosen) != -1]
        EV_df = EV_df[EV_df['출원연도'].isin(years_set)]
        
        
        application_filtered = set(EV_df['등록번호'])
        tmp_df = citation_df.isin(application_filtered)
        citation_df = citation_df[(tmp_df['source'] == True) & (tmp_df['target'] == True)]
        print(str(len(citation_origin)) + " 연결에서 " + str(len(citation_df))+ "개의 연결로")

        if len(citation_df) != 0:
            # 메인패스 작동         
            new_EV_df, node_df = mainpath_findout(EV_df, citation_df, condition_1_name = class_chosen, condition_2_name = year_chosen)

            node_df = node_df[node_df['gp'] != 0]
            node_df['year_chosen'] = year_chosen
            node_df['class_chosen'] = class_chosen
            df_list.append(node_df)
            if year_chosen == 2023:
                new_EV_df['class_chosen'] = class_chosen
                EV_list.append(new_EV_df)

print()
print("iteration completed")
node_df = pd.concat(df_list, ignore_index = True)
EV_df = pd.concat(EV_list, ignore_index = True)

In [ ]:
node_df.to_excel('230811_EV_from2000_yearstack.xlsx')


In [ ]:
EV_df.to_excel('230811_EV_from2000_2023.xlsx')

In [ ]:
del df_list
# del test_df

In [ ]:
node_df = pd.read_excel("230721_EV_yearstack.xlsx")

In [ ]:
node_df = node_df.iloc[:, 1:]
node_df

In [ ]:
EV_df_lst = []
class_list = list(set(EV_df['소분류'].str.replace("\/","")))
print(class_list)

for i in class_list:
    print(i)
    tmp_df = pd.read_excel(fr'230630_소분류 보고\2022_{i}_0.3gp.xlsx',
                          names = ['연번', '출원번호', '출원일', '출원연도', '출원인(정비)', '발명의명칭', '대표청구항', '요약', '대분류', '대분류코드', '중분류', '중분류코드', '소분류', '소분류코드', '국가코드', '특허청', '특허실용구분', '청구항수', '법적상태', '심사청구여부KRJPEP', '심사진행상태', '소멸이유', '분할출원구분', '공지예외주장여부', '정정공보여부', '심판횟수', '출원인', '출원인원문', '출원인국적', '출원인식별기호JP', '현재권리자', '현재권리자국적', '발명자', '발명자원문', '발명자국적', '제1출원인', '제1출원인원문', '제1출원인_정비', '제1출원인_분석', '제1출원인국적', '제1출원인국적_정비', '제1출원인국적_분석', '단독출원여부', '특허고객번호', '출원인수', '발명자수', '현재권리자수', '문헌종류코드', '분석구간', '공개번호', '공개일', '공고번호', '공고일', '등록번호', '등록일', '심사청구일KRJPEP', '존속기간만료일', '원출원번호', '원출원일자', '메인IPC', '전체IPC', '메인CPC', '전체CPC', '우선권국가', '우선권번호', '우선권주장일', '최우선일', '국제출원번호', '국제출원일', '국제공개번호', '국제공개일', '번역문제출일KR', '인용문헌번호', '인용문헌수', '국가별인용문헌수', '인용문헌국가', '자국인용문헌번호', '심사관인용문헌KRUS', '피인용문헌번호', '피인용문헌국가', '피인용문헌수', '피인용문헌수_분석', '국가별피인용문헌수', '비특허문헌수', '패밀리국가수', '패밀리문헌수', '패밀리문헌번호', '패밀리국가별문헌수', 'EPO심플패밀리문헌번호', 'EPO심플패밀리문헌번호_분석', 'EPO심플패밀리문헌수', 'EPO심플패밀리국가별문헌수', 'EPO심플패밀리국가수', '3극출원여부', '가설1', '코드', '임시', 'target_x', 'kp_x', 'target_y', 'kp_y', 'node_type', 'layer', 'lp', 'gp', 'is_HPP', 'is_mainpath'])
#     print(len(tmp_df))
    EV_df_lst.append(tmp_df)
    print(len(EV_df.columns))

tmp_df = pd.concat(EV_df_lst, ignore_index = True)


In [ ]:
node_df = node_df.merge(EV_df[['등록번호', '출원연도']].drop_duplicates(), left_on = 'target', right_on = "등록번호", how = "left")
node_df.drop('등록번호', axis = 1, inplace = True)
node_df

In [ ]:
node_df.to_excel("230724_EV_TP_yearstack.xlsx")

In [ ]:
# pp값 뽑아내기

# 이 기간 동안 'class'별 레코드 수를 계산합니다
# EV_df기준 필요

In [ ]:
import pandas as pd
import numpy as np


pp_df_list = []
next_n = 5

tmp_df = EV_origin[EV_origin['문헌종류 코드'].isin(['B1', 'B2'])].copy()


# class_counts = tmp_df[[class_standard, '출원연도']].value_counts()
class_counts = tmp_df[['출원연도']].value_counts()

class_counts = class_counts.reset_index()
class_counts.columns = ['year_chosen', 'counts']
class_counts = class_counts.sort_values(['year_chosen'])
class_counts = class_counts.reset_index(drop=True)
# 각 분류별로 그룹화하고, n년 동안의 합계를 계산합니다.
result = class_counts['counts'].rolling(next_n).sum().reset_index(drop=True)
class_counts['counts_next_n_years'] = result

# 결과를 출력합니다.
display(class_counts)



# 각 'code'의 최초 등장 연도를 찾습니다
first_year = node_df[['target', '출원연도']].drop_duplicates()
first_year.columns = ['target', 'year_chosen']
first_kp = first_year.merge(node_df, on = ['target', 'year_chosen'], how = 'left')
first_kp = first_kp[['target', 'year_chosen', 'kp']].fillna(0)

first_kp['next_year'] = first_kp['year_chosen'] + 1

# 1+5년 후의 데이터를 가져옵니다
first_kp['year_plus_n'] = first_kp['next_year'] + next_n
first_kp.columns = ['target', 'application_year', 'kp', 'next_year', 'year_plus_n']
first_kp = first_kp[first_kp['application_year'] < (2022 - next_n)]


df_n_yr = pd.merge(first_kp, node_df[['target', 'year_chosen', 'kp']], left_on=['target', 'next_year'], right_on=['target', 'year_chosen'], how='left')
tmp = list(first_kp.columns)
tmp.extend(['next_year_match', 'kp_next_year'])
df_n_yr.columns = tmp
df_n_yr = df_n_yr.fillna(0)

df_n_yr = pd.merge(df_n_yr, node_df[['target', 'year_chosen', 'kp']], left_on=['target', 'year_plus_n'], right_on=['target', 'year_chosen'], how='inner')
tmp.extend(['plus_n_match', 'kp_plus_n'])
df_n_yr.columns = tmp
df_n_yr = df_n_yr.fillna(0)

# kp 변화량을 계산합니다
df_n_yr['kp_change'] = df_n_yr['kp_plus_n'] - df_n_yr['kp_next_year']

# 최종 결과를 출력합니다
print(df_n_yr[['target', 'kp_change']])

# 5년 동안의 count를 계산합니다
tmp_class_counts = class_counts[['year_chosen', 'counts_next_n_years']]
tmp_class_counts.columns = ['end_year', 'count']

df_n_yr = pd.merge(df_n_yr, tmp_class_counts, left_on = 'year_plus_n', right_on = 'end_year', how='inner')
df_n_yr['pp'] = (df_n_yr['kp_change'] / (next_n - 1)) / np.log(df_n_yr['count'])

pp_df_list.append(df_n_yr)

pp_df = pd.concat(pp_df_list, ignore_index = True)


In [ ]:
pp_df

In [ ]:
pp_df.to_excel("230724_EV_pp_5yr.xlsx")

In [ ]:
# node_df의 main path만 뽑고
# 각 연도별 데이터 부여(없다면 해당 년도는 0으로 부여)
# 각 년도별 gp로 순위 부여
# 각 년도별 패러다임 조사
# 그래프로 시각화, 각
pp_df.drop_duplicates(inplace = True)
report_df = pp_df[['target', 'pp']].merge(EV_df, left_on = ['target'], right_on = ['등록번호'], how = 'left')
report_df


In [ ]:
report_df.to_excel("230723_EV_total_pp.xlsx")

In [ ]:
# potential paradigm 점수 구하기
# PP of Pat(i) = 최근 t년 기간의 시작과 끝 점 기준의 KP 기울기 / ln(최근 t년 동안의 특허 사이즈)
# t=...3년?

In [ ]:
# corelation matix로 비교하기
# Let's define a color map for the property
color_map = {'장비·디바이스': 'blue', '플랫폼': 'green', '제조솔루션':'pink',
             '생산현장': 'skyblue', '제어 및 측정장치': 'cyan', '제어시스템':'magenta',
             '개발 및 운영시스템':'skyblue', '비즈니스 시스템':'cyan',
             '데이터플랫폼':'cyan', '인프라':'magenta',
             '3D 프린팅': 'red', '3D\xa0프린팅':'red', 'AR/VR/MR': 'orange', '로봇': 'yellow', '머신비전': 'purple',
             'CNC장비':'red', 'Motion Controller':'orange', '스마트 센서':'purple',
             'DCS':'red','HMI':'orange','PLC':'yellow', 'SCADA':'purple',
             'APS':'red', 'CAx':'orange', 'FEMS':'yellow', 'MES/MOM':'purple', 'PLM':'lime',
             'ERP':'yellow', 'SCM':'purple',
             'Big data/AI':'lime', 'Cloud computing':"purple", 'CPS/Digital twin':"yellow", 'Edge computing':"orange",
             '5G 네트워크':'red', 'IIoT':"lime", '보안':"yellow"

            }  # add more classes and colors if needed

In [ ]:
EV_df = pd.read_excel(r'230722_전체 MainPath\2023_test_tmp_0.3gp.xlsx')
EV_df = EV_df.iloc[:, 1:]
EV_df

In [ ]:


import networkx as nx


# Main Path만 포함
tmp_df = EV_df.copy()
mp_df = tmp_df[tmp_df['is_mainpath'] == True].copy()

citation_df = citation_origin.copy()

# citation_df에서 main path관련 정보만 남김
tmp_df = citation_origin.merge(mp_df, left_on ='source', right_on='등록번호', how='inner')
tmp_df = tmp_df[['source', 'target']]
tmp_df.columns = ['source', 'target']

tmp_df = mp_df.merge(tmp_df, left_on='등록번호', right_on = 'target', how= 'inner')
tmp_df = tmp_df[['source', 'target']]

# # 익명화

# nodes = set(tmp_df['source']).union(set(tmp_df['target']))
# node_mapping = {node: f'N{i}' for i, node in enumerate(nodes)}

# # 데이터프레임의 노드 번호를 새로운 코드로 업데이트하기
# tmp_df['source'] = tmp_df['source'].apply(lambda x: node_mapping[x])
# tmp_df['target'] = tmp_df['target'].apply(lambda x: node_mapping[x])

# networkx 네트워크로 구성
from_to_lst = [(row['source'], row['target']) for _, row in tmp_df.iterrows()]
mp_nx = nx.DiGraph()
mp_nx.add_edges_from(from_to_lst)


# property 추가
import pandas as pd
import networkx as nx

# Suppose we have the following DataFrame

# # Now, add properties to these nodes from the DataFrame
# for idx, row in EV_df.iterrows():
#     node = row['등록번호']
#     if row[class_standard] == class_chosen:
#         if node in mp_nx:
#             mp_nx.nodes[node]['대분류'] = row['대분류']
#             mp_nx.nodes[node]['중분류'] = row['중분류']
#             mp_nx.nodes[node]['소분류'] = row['소분류']
# # !@$!@%!@%%$@!%@! 여러 코드에 분류가 중복되는 경우????

# # Check the properties of the nodes
# for node, data in mp_nx.nodes(data=True):
#     print(f"Node: {node}, Data: {data}")

connected_subgraphs = list(nx.weakly_connected_components(mp_nx))
# 연결된 부분 그래프의 개수 출력
print(f"연결된 부분 그래프의 개수: {len(connected_subgraphs)}")

# 각 연결된 부분 그래프 출력
for i, subgraph in enumerate(connected_subgraphs):
    print(f"연결된 부분 그래프 {i+1}: {subgraph}")

import matplotlib.pyplot as plt
from collections import defaultdict

# print(classification_chosen)


H = mp_nx


# # Get a list of node properties
# chosen_feature = class_standard
# properties = [data[chosen_feature] if chosen_feature in data else 'missing' for node, data in H.nodes(data=True)]
# colors = [color_map[property] if property in color_map else 'gray' for property in properties]

# Figure size adjustment
plt.figure(figsize=(20, 100))

# Find the source nodes (nodes without incoming edges)
source_nodes = [n for n, d in H.in_degree() if d==0]

# Calculate node levels for each source node and BFS tree
levels = defaultdict(list)
for source_node in source_nodes:
    bfs_tree = nx.bfs_tree(H, source_node)
    level = nx.single_source_shortest_path_length(bfs_tree, source_node)
    for node, lv in level.items():
        levels[lv].append(node)

# Create user-defined node positions
pos = {node: (level, i/(len(nodes)-1)) if len(nodes) > 1 else (level, 0.5) for level, nodes in levels.items() for i, node in enumerate(nodes)}

# Draw the graph
nx.draw(H, pos, with_labels=False, node_size=2000, font_size=15, arrowsize=20, edge_color="lightgray")

# Adjust node label positions
label_pos = {k: [v[0], v[1]] for k, v in pos.items()}  # adjust the labels slightly above the nodes

# Draw node labels
nx.draw_networkx_labels(H, label_pos, font_size=15)

try:
    plt.savefig(f"subgraph_{chosen_feature}_{class_chosen}.png")
except:
    plt.savefig(f"subgraph_tmp_.png")

# Display the graph
plt.show()


In [ ]:
# property 추가
import pandas as pd
import networkx as nx

# Suppose we have the following DataFrame

# Now, add properties to these nodes from the DataFrame
for idx, row in EV_df.iterrows():
    node = row['등록번호']
    if node in mp_nx:
        mp_nx.nodes[node]['대분류'] = row['대분류']
        mp_nx.nodes[node]['중분류'] = row['중분류']
        mp_nx.nodes[node]['소분류'] = row['소분류']
# !@$!@%!@%%$@!%@! 여러 코드에 분류가 중복되는 경우????
        
# Check the properties of the nodes
for node, data in mp_nx.nodes(data=True):
    print(f"Node: {node}, Data: {data}")



In [ ]:
# mp_df = EV_df
# edges = citation_df
# tmp_df = edges

In [ ]:
# classification = list(set(EV_df[class_standard]))
class_standard

In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict




for i, subgraph in enumerate(connected_subgraphs):
    H = mp_nx.subgraph(subgraph)

    # Get a list of node properties
    chosen_feature = '대분류'
    properties = [data[chosen_feature] if chosen_feature in data else 'missing' for node, data in H.nodes(data=True)]
    colors = [color_map[property] if property in color_map else 'gray' for property in properties]

    # Figure size adjustment
    plt.figure(figsize=(20, 20))

    # Find the source nodes (nodes without incoming edges)
    source_nodes = [n for n, d in H.in_degree() if d==0]

    # Calculate node levels for each source node and BFS tree
    levels = defaultdict(list)
    for source_node in source_nodes:
        bfs_tree = nx.bfs_tree(H, source_node)
        level = nx.single_source_shortest_path_length(bfs_tree, source_node)
        for node, lv in level.items():
            levels[lv].append(node)

    # Create user-defined node positions
    pos = {node: (level, i/(len(nodes)-1)) if len(nodes) > 1 else (level, 0.5) for level, nodes in levels.items() for i, node in enumerate(nodes)}

    # Draw the graph
    nx.draw(H, pos, with_labels=False, node_size=2000, node_color=colors, font_size=15, arrowsize=20, edge_color="lightgray")

    # Adjust node label positions
    label_pos = {k: [v[0], v[1]] for k, v in pos.items()}  # adjust the labels slightly above the nodes

    # Draw node labels
    nx.draw_networkx_labels(H, label_pos, font_size=15)

    # Save the graph as a .png file
    if 'classification_chosen' not in locals():
        classification_chosen = "tmp"
    plt.savefig(f"subgraph_{chosen_feature}_{i}.png")

    # Display the graph
    plt.show()


In [ ]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

# explode(df.assign(var1=df.var1.str.split(',')), 'var1')

In [ ]:

for class_chosen in classes_list:
    print(class_chosen.replace("/", ""))

In [ ]:
import numpy as np 

assignee_list = []

for class_chosen in classes_list:
    print(class_chosen)
    class_chosen_df = EV_df[EV_df[class_standard] == class_chosen]
    #     print(len(class_chosen_df))

    df = class_chosen_df

    assignee_df = df.loc[:,['등록번호','출원인(정비)']]
    assignee_df.columns = ['PatNum', 'assignee']
    assignee_df = assignee_df[assignee_df['assignee'] != " "]

    assignee_df = explode(assignee_df.assign(assignee = assignee_df.assignee.str.split("|")),'assignee')

    top_n = 30
    top_n_assignee_df = assignee_df.groupby('assignee').count().sort_values('PatNum', ascending = False).iloc[0:top_n,:]
    
    top_n_assignee_df.to_excel('mp_top_'+str(top_n)+"_list_of_"+class_chosen.replace("/","")+".xlsx")
    
    top_n_assignee_df = top_n_assignee_df.reset_index()
    assignee_df = assignee_df[assignee_df['assignee'].isin(set(top_n_assignee_df['assignee']))]
    assignee_df = assignee_df.merge(EV_df[['등록번호', '출원연도', class_standard, '국가코드']].drop_duplicates(), left_on = 'PatNum', right_on = '등록번호')

#     display(assignee_df)
    assignee_list.append(assignee_df)

assignee_df = pd.concat(assignee_list,ignore_index = True)
assignee_df.to_excel(f"{class_standard}_assignee_기초통계.xlsx")
    
    # class_chosen_df
assignee_df

In [ ]:
# class_chosen_df = class_chosen_df[class_chosen_df['is_mainpath'] == True].copy()

In [ ]:
classes = list(set(EV_df[class_standard]))

tmp_lst = []

for class_chosen in classes:
    tmp_df = EV_df[EV_df[class_standard] == class_chosen].copy()
    tmp_df = tmp_df[['등록번호', '출원연도', '국가코드']]
    tmp_df.drop_duplicates(inplace = True)
    tmp_df = tmp_df.groupby('출원연도').count()
    tmp_df['class'] = class_chosen
    tmp_df.reset_index(inplace = True)
    
    tmp_lst.append(tmp_df)
    
count_df = pd.concat(tmp_lst, ignore_index = True)
count_df.columns = ['출원연도', '출원건수', '출원건수2', 'class']
count_df = count_df[['출원연도', '출원건수', 'class']]
count_df

In [ ]:
import numpy as np 

df = EV_df.copy()

assignee_df = df.loc[:,['등록번호','출원인(정비)']]
assignee_df.columns = ['PatNum', 'assignee']
assignee_df = assignee_df[assignee_df['assignee'] != " "]

assignee_df = explode(assignee_df.assign(assignee = assignee_df.assignee.str.split("|")),'assignee')
assignee_df

assignee_df = assignee_df.merge(EV_df[['등록번호', '출원연도', class_standard]].drop_duplicates(), left_on = "PatNum", right_on = '등록번호', how = "left")
# assignee_df
tmp_df = assignee_df.groupby(['출원연도', class_standard]).count()
tmp_df.reset_index(inplace= True)
tmp_df = tmp_df.iloc[:, :3]
tmp_df.columns = ['출원연도', 'class', '출원인수']
tmp_df

In [ ]:
count_df = count_df.merge(tmp_df, on = ['출원연도', 'class'], how = 'left')
count_df

count_df.to_excel(f'연도별 통계_{class_standard}_.xlsx')

In [ ]:


# classification_chosen기준으로 저장
# top_n_assignee_df.to_excel('top_'+str(top_n)+"_list_of_"+classification_chosen+".xlsx")

In [ ]:
EV_df[['등록번호', '출원연도']].drop_duplicates()

In [ ]:
import numpy as np 

total_df = pd.DataFrame()

for class_chosen in set(EV_df['중분류']):
    print(class_chosen)
    df = EV_df.copy()
    df = df[df['중분류'] == class_chosen]
    
    
    assignee_df = df.loc[:,['등록번호','출원인(정비)']]
    assignee_df.columns = ['PatNum', 'assignee']
    assignee_df = assignee_df[assignee_df['assignee'] != " "]

    assignee_df = explode(assignee_df.assign(assignee = assignee_df.assignee.str.split("|")),'assignee')

    top_n = 15
    top_n_assignee_df = assignee_df.groupby('assignee').count().sort_values('PatNum', ascending = False).iloc[0:top_n,:]
    top_n_assignee_df['class'] = class_chosen
    
    total_df = pd.concat([total_df, top_n_assignee_df])
total_df.reset_index(inplace=True)    
    
total_df.to_excel('top_'+str(top_n)+"_list_of_"+"classes.xlsx")

In [ ]:
top_assignees = total_df.groupby('class').apply(lambda x: x.nlargest(5, 'PatNum')['assignee'].tolist()).reset_index()

# 출원인 리스트를 문자열로 변환
top_assignees['top_5'] = top_assignees[0].apply(lambda x: ', '.join(x))
top_assignees.drop(0, axis=1, inplace=True)

print(top_assignees)

In [ ]:
top_assignees

In [ ]:
# Assignee를 기준으로 연도별 출원 동향 보기56
Annual_df = df[['등록번호', '등록일', '출원인']]
Annual_df = Annual_df[Annual_df['등록일'] >'1990-01-01']
Annual_df.loc[:, '등록일'] = pd.to_datetime(Annual_df['등록일']).dt.to_period('Y')
Annual_df.columns = ['등록번호', '등록년도', 'assignee']



Annual_df = Annual_df[Annual_df['assignee'] != " "]

# Annual_df.assignee.str.split("|")
# 
Annual_df = explode(Annual_df.assign(assignee = Annual_df.assignee.str.split("|")),'assignee')



Annual_df = Annual_df.groupby(['등록년도', 'assignee']).count()
Annual_df.reset_index(inplace = True)

Annual_df.columns = ['등록년도', 'assignee', 'count']




# Annual_df




In [ ]:
Annual_pv = Annual_df.groupby(['assignee', '등록년도']).sum('count')

In [ ]:
Annual_pv

In [ ]:
set_size = [20,15]
# plot = Annual_pv.plot(figsize = set_size, style = 'o-')


# for p in plot.patches:
#     plot.annotate(str(p.get_height()), (p.get_x() * 0.999, p.get_height() * 1.005))
# fig = plot.get_figure()
# fig.savefig(directory_path + "\\EDA\\" + chosen_class +"_line_" + search_class + ".png")


import matplotlib.pyplot as plt
#df.index = pd.to_datetime(df.index)  # If not a DatetimeIndex
pts = Annual_pv.resample('1Y').asfreq()
# pts
fig, ax = plt.subplots(figsize = set_size)

Annual_pv.plot(kind='line', ax=ax)
# pts.plot(marker='o', ax=ax, lw=0, legend=False)

pad = 1
for idx, val in pts.stack().iteritems():
    ax.annotate(val, (idx[0], val+pad))
# https://stackoverflow.com/questions/55972583/plotting-multiple-lines-want-a-mark-every-with-a-text-label-on-each-line
ax.grid(axis='y')